# Алгоритмы интеллектуальной обработки больших объемов данных
## Домашнее задание №2 - Дерево решений


**Общая информация**

**Срок сдачи:** до 27 марта 2018, 06:00   
**Штраф за опоздание:** -2 балла после 06:00 27 марта, -4 балла после 06:00 3 апреля, -6 баллов после 06:00 10 апреля

При отправлении ДЗ указывайте фамилию в названии файла   


Присылать ДЗ необходимо в виде ссылки на свой github репозиторий в slack @alkhamush
Необходимо в slack создать таск в приватный чат:   
/todo Фамилия Имя *ссылка на гитхаб* @alkhamush   
Пример:   
/todo Ксения Стройкова https://github.com/stroykova/spheremailru/stroykova_hw2.ipynb @alkhamush   

Используйте данный Ipython Notebook при оформлении домашнего задания.

###### Задание 1 (2 баллов)
Разберитесь в коде MyDecisionTreeClassifier, который уже частично реализован. В комментариях, где написано "Что делает этот блок кода?", ответьте на этот вопрос. Допишите код там, где написано "Ваш код". Ваша реализация дерева должна работать по точности не хуже DecisionTreeClassifier из sklearn. Точность проверяется на wine и Speed Dating Data.

###### Задание 2 (2 балла)
Добиться скорости работы на fit сравнимой со sklearn wine и Speed Dating Data. 
Для этого используем numpy. 

###### Задание 3 (2 балла)
Продемонстрируйте умение работать с Pipeline на данных Speed Dating Data и DecisionTreeClassifier. Нужно в pipeline произвести все необходимые преобразования данных и в конце обучить модель. Задание реализуйте под пунктом Задание 3 (уже написано ниже)

###### Задание 4 (2 балла)
Добавьте функционал, который определяет значения feature importance. Выведите 10 главных фичей под пунктом Задание 4 (уже написано ниже) для MyDecisionTreeClassifier и DecisionTreeClassifier так, чтобы сразу были видны выводы и по MyDecisionTreeClassifier, и по DecisionTreeClassifier. Используем данные Speed Dating Data.

###### Задание 5 (2 балла)
С помощью GridSearchCV или RandomSearchCV подберите наиболее оптимальные параметры для случайного леса (Выберете 2-3 параметра). Используем данные Speed Dating Data. Задание реализуйте под пунктом Задание 5 (уже написано ниже)


**Штрафные баллы:**

1. Невыполнение PEP8 -1 балл
2. Отсутствие фамилии в имени скрипта (скрипт должен называться по аналогии со stroykova_hw2.ipynb) -1 балл
3. Все строчки должны быть выполнены. Нужно, чтобы output команды можно было увидеть уже в git'е. В противном случае -1 балл
4. При оформлении ДЗ нужно пользоваться данным файлом в качестве шаблона. Не нужно удалять и видоизменять написанный код и текст. В противном случае -1 балл

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.datasets import load_wine
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import KFold, train_test_split, \
    GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline

%matplotlib inline


In [2]:
class MyDecisionTreeClassifier:
    NON_LEAF_TYPE = 0
    LEAF_TYPE = 1

    def __init__(self, min_samples_split=2, max_depth=None,
                 sufficient_share=1.0, criterion='gini', max_features=None):
        self.tree = dict()
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.sufficient_share = sufficient_share
        self.num_class = -1
        self.feature_importances_ = None
        if criterion == 'gini':
            self.G_function = self.__gini
        elif criterion == 'entropy':
            self.G_function = self.__entropy
        elif criterion == 'misclass':
            self.G_function = self.__misclass
        else:
            print 'invalid criterion name'
            raise

        if max_features == 'sqrt':
            self.get_feature_ids = self.__get_feature_ids_sqrt
        elif max_features == 'log2':
            self.get_feature_ids = self.__get_feature_ids_log2
        elif max_features is None:
            self.get_feature_ids = self.__get_feature_ids_N
        else:
            print 'invalid max_features name'
            raise

    def __gini(self, l_c, l_s, r_c, r_s):
        l_s = l_s.astype('float')
        r_s = r_s.astype('float')
        return 1 - ((l_c ** 2 / l_s).sum(axis=1) +
                    (r_c ** 2 / r_s).sum(axis=1)) / (l_s + r_s)[0]

    def __gini_gain(self, y):
        return 1 - np.sum((np.bincount(y).astype(float) / y.size) ** 2)

    def __entropy(self, l_c, l_s, r_c, r_s):
        return -(((l_c / l_s).log2() * l_c).nansum(axis=1) +
                 ((r_c / r_s).log2() * r_c).nansum(axis=1)) / (l_s + r_s)[0]

    def __entropy_gain(self, y):
        p = np.bincount(y).astype(float) / y.size
        return -np.nansum(p * np.log2(p))

    def __misclass(self, l_c, l_s, r_c, r_s):
        return 1 - (l_c.max(axis=1) + r_c.max(axis=1)) / (l_s + r_s)[0]

    def __misclass_gain(self, y):
        return 1 - np.max(np.bincount(y).astype(float) / y.size)

    def __get_feature_ids_sqrt(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        return np.array(feature_ids[:int(np.sqrt(n_feature))])

    def __get_feature_ids_log2(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        return np.array(feature_ids[:int(np.log2(n_feature))])

    def __get_feature_ids_N(self, n_feature):
        return np.arange(n_feature)

    def __sort_samples(self, x, y):
        sorted_idx = x.argsort()
        return x[sorted_idx], y[sorted_idx]

    def __div_samples(self, x, y, feature_id, threshold):
        left_mask = x[:, feature_id] > threshold
        right_mask = ~left_mask
        return x[left_mask], x[right_mask], y[left_mask], y[right_mask]

    def __find_threshold(self, x, y):
        # Сортируем x и y по возрастанию x
        sorted_x, sorted_y = self.__sort_samples(x, y)
        # Считаем количество различных классов в y
        class_number = np.unique(y).shape[0]

        # Разбиваем классы так, чтобы в вершинах было хотя бы минимальное
        # число (min_samples_split) классов
        splitted_sorted_y = \
            sorted_y[self.min_samples_split:-self.min_samples_split]
        # r_border_ids — индексы элементов, не совпадающих с следующим за
        # ними элементом, к которым добавили min_samples_split + 1. По сути
        # r_border_ids — предел, при котором класс меняется
        r_border_ids = \
            np.where(splitted_sorted_y[:-1] != splitted_sorted_y[1:])[0] + \
            (self.min_samples_split + 1)

        if len(r_border_ids) == 0:
            return float('+inf'), None

        # Считаем для каждого интервала одного класса число элементов
        eq_el_count = r_border_ids - np.append([self.min_samples_split],
                                               r_border_ids[:-1])
        # one_hot_code заполнили нулями, а затем проставили 1 там, где
        # начинается новый класс, т. е. записали разбиение через one hot
        # encoding
        one_hot_code = np.zeros((r_border_ids.shape[0], self.num_class))
        one_hot_code[np.arange(r_border_ids.shape[0]),
                     sorted_y[r_border_ids - 1]] = 1
        # class_increments — матрица, в которой описано соответствие между
        # классами и интервалами
        class_increments = one_hot_code * eq_el_count.reshape(-1, 1)
        # Добавляем в первую строку откинутые элементы из y
        class_increments[0] = class_increments[0] + \
            np.bincount(sorted_y[:self.min_samples_split],
                        minlength=self.num_class)

        # Находим количество классов в левой и правой частях разбиения
        l_class_count = np.cumsum(class_increments, axis=0)
        r_class_count = np.bincount(sorted_y, minlength=self.num_class) - \
            l_class_count
        # Находим количество элементов в левой и правой частях разбиения
        l_sizes = r_border_ids.reshape(l_class_count.shape[0], 1)
        r_sizes = sorted_y.shape[0] - l_sizes

        # Посчитаем неопределенность для каждого разбиения
        gs = self.G_function(l_class_count, l_sizes, r_class_count, r_sizes)
        # Выбираем минимум из неопределенностей
        idx = np.argmin(gs)

        # Возвращаем наименьшую неопределенность и порог для признака
        left_el_id = l_sizes[idx][0]
        return gs[idx], (sorted_x[left_el_id-1] + sorted_x[left_el_id]) / 2.0

    def __fit_node(self, x, y, node_id, depth, pred_f=-1):
        if (self.max_depth is not None and depth >= self.max_depth) or \
                y.shape[0] < self.min_samples_split or\
                self.sufficient_share <= float(np.bincount(y).max()) / y.size:
            self.tree[node_id] = (self.LEAF_TYPE, np.bincount(y).argmax(),
                                  np.bincount(y).astype(float) / y.size)
            return

        feature_ids = self.get_feature_ids(x.shape[1])
        thresholds = np.array([self.__find_threshold(x[:, feature_id], y)
                               for feature_id in feature_ids])
        best_ts_id = np.argmin(thresholds[:, 0])
        _, best_ts = thresholds[best_ts_id]
        if best_ts is None:
            self.tree[node_id] = (self.LEAF_TYPE, np.bincount(y).argmax(),
                                  np.bincount(y).astype(float) / y.size)
            return

        if self.G_function == self.__gini:
            gain_func = self.__gini_gain
        elif self.G_function == self.__entropy:
            gain_func = self.__entropy_gain
        else:
            gain_func = self.__misclass_gain

        x_l, x_r, y_l, y_r = \
            self.__div_samples(x, y, feature_ids[best_ts_id], best_ts)
        if x_l.shape[0] == 0 or x_r.shape[0] == 0:
            self.tree[node_id] = (self.LEAF_TYPE, np.bincount(y).argmax(),
                                  np.bincount(y).astype(float) / y.size)
            return

        self.tree[node_id] = (self.NON_LEAF_TYPE, feature_ids[best_ts_id],
                              best_ts)
        self.feature_importances_[best_ts_id] += gain_func(y) - \
            (gain_func(y_l) * y_l.size + gain_func(y_r) * y_r.size) / y.size
        self.__fit_node(x_l, y_l, 2 * node_id + 1, depth + 1, pred_f)
        self.__fit_node(x_r, y_r, 2 * node_id + 2, depth + 1, pred_f)

    def fit(self, x, y):
        self.num_class = np.unique(y).size
        self.feature_importances_ = np.zeros(x.shape[1])
        self.__fit_node(x, y, 0, 0)
        self.feature_importances_ = \
            self.feature_importances_.astype(float) / y.size

    def __predict_class(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_class(x, 2 * node_id + 1)
            else:
                return self.__predict_class(x, 2 * node_id + 2)
        else:
            return node[1]

    def __predict_probs(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_probs(x, 2 * node_id + 1)
            else:
                return self.__predict_probs(x, 2 * node_id + 2)
        else:
            return node[2]

    def predict(self, X):
        return np.array([self.__predict_class(x, 0) for x in X])

    def predict_probs(self, X):
        return np.array([self.__predict_probs(x, 0) for x in X])

    def fit_predict(self, x_train, y_train, predicted_x):
        self.fit(x_train, y_train)
        return self.predict(predicted_x)


In [3]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)


In [4]:
wine = load_wine()
X_train, X_test, y_train, y_test = \
    train_test_split(wine.data, wine.target, test_size=0.1,
                     stratify=wine.target)


## Проверка скорости работы на wine

In [5]:
%time clf.fit(X_train, y_train)


CPU times: user 1.66 ms, sys: 388 µs, total: 2.05 ms
Wall time: 1.82 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [6]:
%time my_clf.fit(X_train, y_train)


CPU times: user 21.7 ms, sys: 1.09 ms, total: 22.8 ms
Wall time: 23.9 ms


## Проверка качества работы на wine

In [7]:
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')


0.88850038850038848

In [8]:
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')


0.89177489177489166

## Подготовка данных Speed Dating Data 

In [9]:
def prepare(df):
    global X, y, features_names
    df = df.iloc[:, :97]
    df = df.drop(['id'], axis=1)
    df = df.drop(['idg'], axis=1)
    df = df.drop(['condtn'], axis=1)
    df = df.drop(['round'], axis=1)
    df = df.drop(['position', 'positin1'], axis=1)
    df = df.drop(['order'], axis=1)
    df = df.drop(['partner'], axis=1)
    df = df.drop(['age_o', 'race_o', 'pf_o_att', 'pf_o_sin', 'pf_o_int',
                  'pf_o_fun', 'pf_o_amb', 'pf_o_sha', 'dec_o', 'attr_o',
                  'sinc_o', 'intel_o', 'fun_o', 'amb_o', 'shar_o', 'like_o',
                  'prob_o', 'met_o'], axis=1)
    df.drop_duplicates('iid').age.isnull().sum()
    df = df.dropna(subset=['age'])
    df.loc[:, 'field_cd'] = df.loc[:, 'field_cd'].fillna(0)
    df = df.drop(['field'], axis=1)
    pd.get_dummies(df, columns=['field_cd'], prefix='field_cd', prefix_sep='=')
    df = df.drop(['undergra'], axis=1)
    df.loc[:, 'mn_sat'] = \
        df.loc[:, 'mn_sat'].str.replace(',', '').astype(np.float)
    df.loc[:, 'mn_sat'] = df.mn_sat.fillna(-999)
    df.loc[:, 'tuition'] = \
        df.loc[:, 'tuition'].str.replace(',', '').astype(np.float)
    df.loc[:, 'tuition'] = df.tuition.fillna(-999)
    df = df.dropna(subset=['imprelig', 'imprace'])
    df = df.drop(['from', 'zipcode'], axis=1)
    df.loc[:, 'income'] = \
        df.loc[:, 'income'].str.replace(',', '').astype(np.float)
    df.loc[:, 'income'] = df.loc[:, 'income'].fillna(-999)
    df = df.dropna(subset=['date'])
    df.loc[:, 'career_c'] = df.loc[:, 'career_c'].fillna(0)
    df = df.drop(['career'], axis=1)
    df = df.drop(['sports', 'tvsports', 'exercise', 'dining', 'museums',
                  'art', 'hiking', 'gaming', 'clubbing', 'reading', 'tv',
                  'theater', 'movies', 'concerts', 'music', 'shopping',
                  'yoga'], axis=1)
    df = df.drop(['expnum'], axis=1)
    feat = ['iid', 'wave', 'attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1',
            'amb1_1', 'shar1_1']
    temp = df.drop_duplicates(subset=['iid', 'wave']).loc[:, feat]
    temp.loc[:, 'totalsum'] = temp.iloc[:, 2:].sum(axis=1)
    df.loc[:, 'temp_totalsum'] = \
        df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1', 'amb1_1',
                   'shar1_1']].sum(axis=1)
    df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1', 'amb1_1',
               'shar1_1']] = (df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1',
                                         'fun1_1', 'amb1_1', 'shar1_1']].T /
                              df.loc[:, 'temp_totalsum'].T).T * 100
    feat = ['iid', 'wave', 'attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1',
            'amb2_1', 'shar2_1']
    temp = df.drop_duplicates(subset=['iid', 'wave']).loc[:, feat]
    temp.loc[:, 'totalsum'] = temp.iloc[:, 2:].sum(axis=1)
    df.loc[:, 'temp_totalsum'] = df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1',
                                            'fun2_1', 'amb2_1', 'shar2_1']]\
                                   .sum(axis=1)
    df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1', 'amb2_1',
               'shar2_1']] = (df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1',
                                         'fun2_1', 'amb2_1', 'shar2_1']].T /
                              df.loc[:, 'temp_totalsum'].T).T * 100
    df = df.drop(['temp_totalsum'], axis=1)

    for i in [4, 5]:
        feat = ['attr{}_1'.format(i), 'sinc{}_1'.format(i),
                'intel{}_1'.format(i), 'fun{}_1'.format(i),
                'amb{}_1'.format(i), 'shar{}_1'.format(i)]
        if i != 4:
            feat.remove('shar{}_1'.format(i))
        df = df.drop(feat, axis=1)

    df = df.drop(['wave'], axis=1)
    df_male = df.query('gender == 1').drop_duplicates(subset=['iid', 'pid'])\
                .drop(['gender'], axis=1).dropna()
    df_female = df.query('gender == 0').drop_duplicates(subset=['iid'])\
                  .drop(['gender', 'match', 'int_corr', 'samerace'], axis=1)\
                  .dropna()

    df_female.columns = df_female.columns + '_f'
    df_female = df_female.drop(['pid_f'], axis=1)
    df_pair = df_male.join(df_female.set_index('iid_f'), on='pid', how='inner')
    df_pair = df_pair.drop(['iid', 'pid'], axis=1)
    X = df_pair.iloc[:, 1:].values
    y = df_pair.iloc[:, 0].values
    features_names = df_pair.columns[1:]


def split_data():
    global X, y, X_train, X_test, y_train, y_test
    X_train, X_test, y_train, y_test = \
        train_test_split(X, y, test_size=0.1, random_state=179179179)


In [10]:
df = pd.read_csv('Speed Dating Data.csv', encoding='cp1251')
prepare(df)
split_data()
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)


## Проверка скорости работы на Speed Dating Data 

In [11]:
%time clf.fit(X_train, y_train)


CPU times: user 87.9 ms, sys: 1.94 ms, total: 89.9 ms
Wall time: 117 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [12]:
%time my_clf.fit(X_train, y_train)


CPU times: user 1.69 s, sys: 24.8 ms, total: 1.71 s
Wall time: 1.85 s


## Проверка качества работы на Speed Dating Data

In [13]:
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')


0.60621674835574546

In [14]:
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')


0.59481103166496422

## Задание 3

In [15]:
df = pd.read_csv('Speed Dating Data.csv', encoding='cp1251')

pipeline = Pipeline([
    ('prepare', prepare(df)),
    ('split', split_data()),
    ('clf', DecisionTreeClassifier()),
])

pipeline.fit(X_train, y_train)
f1_score(y_pred=pipeline.predict(X_test), y_true=y_test, average='macro')


0.62124097998454497

## Задание 4

In [16]:
def get_stat(names, clf):
    return pd.Series(index=features_names, data=clf.feature_importances_)\
             .sort_values(ascending=False)[:10]

print 'my clf\n\n{}\n\nsklearn clf\n\n{}'.format(
    get_stat(features_names, my_clf), get_stat(features_names, clf))


my clf

int_corr      0.000311
age           0.000206
goal_f        0.000146
field_cd      0.000143
sinc2_1_f     0.000136
imprelig      0.000136
intel2_1      0.000132
income_f      0.000117
tuition_f     0.000108
intel1_1_f    0.000088
dtype: float64

sklearn clf

int_corr      0.090542
fun1_1_f      0.036652
amb1_1_f      0.036318
age_f         0.031568
imprelig_f    0.029123
income_f      0.028719
age           0.027012
sinc2_1_f     0.024813
field_cd_f    0.023786
fun2_1_f      0.023254
dtype: float64


## Задание 5

In [17]:
clf = RandomForestClassifier(random_state=179179179)
param_dist = {"max_depth": range(1, 11) + [None],
              "min_samples_split": range(2, 11),
              "n_estimators": range(1, 101),
              "criterion": ["gini", "entropy"]}
n_iter_search = 20
random_search = RandomizedSearchCV(clf, param_distributions=param_dist,
                                   n_iter=n_iter_search)
random_search.fit(X, y)
print random_search.best_params_


{'n_estimators': 77, 'min_samples_split': 7, 'criterion': 'entropy', 'max_depth': 1}
